# pym2v

## Setup

In [ ]:
from datetime import datetime, timedelta

import polars as pl

from pym2v.api import EurogardAPI

## Prototyping

Create the API object

In [ ]:
api = EurogardAPI()

Get user info

In [ ]:
info = api.get_user_info()
info

Get all machines within the account

In [ ]:
machines = api.get_machines()
machines

In [ ]:
len(machines["entities"])

### Get measurements

Get the UUID of the machine you are interested in, e.g. **Aquakultur**

In [ ]:
MACHINE_NAME = "Aquakultur"

In [ ]:
machine_uuid = api.get_machine_uuid(machine_name=MACHINE_NAME, machines=machines)
machine_uuid

Get name of measurements (**not** the actual values) for the selected machine

In [ ]:
result = api.get_machine_measurement_names(machine_uuid)
result

In [ ]:
measurement_names_df = pl.DataFrame(data=result["entities"])

In [ ]:
measurement_names_df.glimpse()

In [ ]:
measurement_names_df.get_column(name="name")

### Get actual data

In [ ]:
START_DATE = datetime(2025, 1, 1)
END_DATE = datetime(2026, 1, 1)
INTERVAL = timedelta(seconds=60)
MAX_FRAME_LENGTH = timedelta(days=30)

In [ ]:
NAMES = [col.strip() for col in measurement_names_df.get_column("name").to_list()]
NAMES

#### Synchronously

In [ ]:
data_df = api.get_long_frame_from_names(
    machine_uuid=machine_uuid,
    names=NAMES,
    start=START_DATE,
    end=END_DATE,
    interval=INTERVAL,
    max_frame_length=MAX_FRAME_LENGTH,
)

In [ ]:
data_df.glimpse()

In [ ]:
data_df.head()

#### Asynchronously

In [ ]:
data_df2 = await api.aget_frame_from_names(
    machine_uuid=machine_uuid,
    names=NAMES,
    start=START_DATE,
    end=END_DATE,
    interval=INTERVAL,
    max_frame_length=MAX_FRAME_LENGTH,
)

In [ ]:
data_df2.glimpse()

In [ ]:
data_df.equals(data_df2)